In [1]:
import tensorflow as tf
import os

In [2]:
checkpoint_path = 'training1/.cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path, save_weights_only = True, verbose = 1)

In [3]:
class myCallback(tf.keras.callbacks.Callback):
    def set_model(self, model):
        self.model = model
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > 0.99:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

In [4]:
callbacks = myCallback()

<font size = '5'> ADDING THE IMAGES TO TENSORFLOW DATASETS (TENSORFLOW INPUT PIPELINING)

<font size = '3'> Creating constant for items that will be used a lot in the program

In [5]:
img_size = 256
batch_size = 64
epochs = 25

<font size = '3'> Adding files to the dataset

In [6]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Data",
    shuffle=True,
    image_size=(img_size,img_size),
    batch_size= batch_size,
    label_mode='int'
)

Found 20638 files belonging to 15 classes.


In [7]:
n_classes = len(dataset.class_names)
n_classes

15

<font size = '3'> This returns a PrefetchDataset

<font size='3'> The folders present in the 'Data'  folder are basically all the classes that are present here.

<font size = '5'> SPLITTING THE DATA INTO TRAIN AND TEST SETS

<font size = '3'> We will split the training set into Train and test set, where the training set is 80% of the dataset and then the remaining 20% is used as 10% for testing the model, and 10% for validation

<font size='3'> We will split the dataset using the '.skip' and the '.take' functions present in the tensorflow

In [8]:
train_len = int(0.8 * len(dataset))
train_set = dataset.take(train_len)
verify_set = dataset.skip(train_len)
print(f"Length of training set = {len(train_set)} elements")

Length of training set = 258 elements


<font size = '3'> Furthur splitting the 'test_set' into test_set and validation_set

In [9]:
test_len = int(0.1*len(dataset))
test_set = verify_set.take(test_len)
validation_set = verify_set.skip(test_len)
print(f"Length of test set = {len(test_set)} elements")
print(f"Length of validation set = {len(validation_set)} elements")

Length of test set = 32 elements
Length of validation set = 33 elements


<font size = '3'> Now we have our train_set, test_set and validation_set

<font size = '3'> Now we will use the '.prefetch' method, because The .prefetch method is used to overlap the data loading and model training. It prefetches batches of data in the background while the model is executing training steps, which makes the program solving faster

<font size ='3'>The '.cache' method is used to cache data, which can significantly improve the data loading speed. We have done mapping before the caching process because:
Caching is done to store the preprocessed data in memory or on disk

In [10]:
train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_set = test_set.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_set = validation_set.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

<font size = '5'> DATA PREPROCESSING

<font size = '3'> Here we will build layers for the preprocessing that has to be done to the images such as Scaling, Resizing and Data Augmentation before the images are passed as inputs

In [11]:
rescale_and_resize = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(img_size,img_size), #this will make sure that all the images are in the required scale
    tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [12]:
data_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

<font size = '5'> BUILDING THE MODEL

<font size = '3'> First we will add the rescale, resize and data augmentation layers in the model

<font size = '3'> Now we will add the CNN layers

<font size = '3'> Creating a stack of convolutional layers

In [13]:
model = tf.keras.Sequential([
    rescale_and_resize,
    data_augmentation,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3),padding='valid'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',padding='valid'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',padding='valid'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',padding='valid'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',padding='valid'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',padding='valid'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=n_classes, activation='softmax')
])


In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [15]:
model.build(input_shape=(None,img_size,img_size,3))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                 

In [16]:
model.fit(x=train_set,epochs=epochs,batch_size=batch_size,validation_data=validation_set,callbacks=[callbacks,cp_callback],verbose=1)

Epoch 1/25


258/258 [==============================] - ETA: 0s - loss: 2.0914 - accuracy: 0.3242
Epoch 1: saving model to training1\.cp.ckpt
258/258 [==============================] - 662s 2s/step - loss: 2.0914 - accuracy: 0.3242 - val_loss: 1.6651 - val_accuracy: 0.4591
Epoch 2/25
258/258 [==============================] - ETA: 0s - loss: 1.2660 - accuracy: 0.5796
Epoch 2: saving model to training1\.cp.ckpt
258/258 [==============================] - 578s 2s/step - loss: 1.2660 - accuracy: 0.5796 - val_loss: 1.1828 - val_accuracy: 0.6044
Epoch 3/25
258/258 [==============================] - ETA: 0s - loss: 0.8986 - accuracy: 0.6949
Epoch 3: saving model to training1\.cp.ckpt
258/258 [==============================] - 619s 2s/step - loss: 0.8986 - accuracy: 0.6949 - val_loss: 0.7964 - val_accuracy: 0.7267
Epoch 4/25
258/258 [==============================] - ETA: 0s - loss: 0.7253 - accuracy: 0.7482
Epoch 4: saving model to training1\.cp.ckpt
258/258 [==============================] -

In [17]:
model.evaluate(test_set)

32/32 [==============================] - 35s 551ms/step - loss: 0.2168 - accuracy: 0.9248


[0.2167801856994629, 0.9248046875]

In [18]:
model.save('/Versions/1/')

INFO:tensorflow:Assets written to: /Versions/1/assets


INFO:tensorflow:Assets written to: /Versions/1/assets
